In [12]:
import pandas as pd                     # pip install pandas
import re                               # for regex
import nltk                             # pip install nltk

from nltk.corpus import stopwords                                       # pip install nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer             # Vader Sentiment Analysis (eng)
from sklearn.feature_extraction.text import CountVectorizer             # pip install scikit-learn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('Reviews.csv')

In [ ]:
# Informação dos atributos
df.info()

In [ ]:
# Mostrar os 5 primeiros dados do arquivo
df.head()

### Analisar os dados

In [ ]:
# Visão geral sobre as avaliações
print(df['Score'].value_counts().sort_index())

# Visualização dos dados
graph = df['Score'].value_counts().sort_index().plot(kind='bar', title='Quantidade de avaliações em cada nota', figsize=(10, 5), color='red')
graph.set_xlabel('Notas')
graph.set_ylabel('Quantidade de avaliações')
graph.plot()

### Classificar os tweets

Classificação levará em conta a nota "Score" que cada um levou na avaliação.
Neutros não serão considerados para o treinamento do modelo.

Score > 3: Positivo

Score < 3: Negativo

Score = 3: Neutro

In [ ]:
# Remove os neutros
df = df[df['Score'] != 3]

# Cria nova coluna "Sentimento" e indica se os dados em positivos, negativos e neutros
df['Sentiment'] = df['Score'].apply(lambda score: 'Positive' if score > 3 else ('Negative' if score < 3 else 'Neutral'))

df.head()


### Limpeza dos dados

Realizando remoção de: Tags HTML, caracteres especiais, colocando todas as letras em minúsculo...

Remoção de Stopwords. Stopwords são paavras que não trazem significado para nossa frase, como "eu, estão, tudo, meu, seu, nosso, deles, estava...". Foram incluidas palavras "like", "good" e "great", pois estavam sendo usadas tanto nas sentenças "negativas" quanto nas positivas, provavelmente com o sentido do produto estar "not good".

Normalização de palavras semelhantes/variações usando "stemming". Stemming é a técnica para remoção de sufixos e prefixos de uma palavra. Ex: Watch, Watched, Watching

##### Remover Tags HTML com Regex

In [ ]:
def remove_tags(text):
    TAG_RE = re.compile(r'<[^>]+>')
    return TAG_RE.sub('', text)

# Remover Tags de todos os 'Reviews'
df.Text = df.Text.apply(remove_tags)
df.Text[0]

##### Remover caracteres especiais (pontuação)

In [ ]:
def remove_special_characters(text):
    # Padrão a ser mantido
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

# Remover caracteres especiais de todos os 'Reviews'
df.Text = df.Text.apply(remove_special_characters)
df.Text[0]

##### Colocar todas as letras em minúsculo

In [ ]:
def lower_case(text):
    return text.lower()

# Colocar todas as letras em minúsculo de todos os 'Reviews'
df.Text = df.Text.apply(lower_case)
df.Text[0]

##### Remover palavras 'irrelevantes' (stopwords)
Tempo médio de execução: 1min

In [13]:
nltk.download('stopwords')
nltk.download('punkt')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
   
    # Adicionar palavras específicas que podem não conter na biblioteca
    stop_words.update(['rt'])
    
    words = word_tokenize(text)
    filtered_text = [word for word in words if word not in stop_words]
    
    return filtered_text

# Remover palavras 'irrelevantes'
df.Text = df.Text.apply(remove_stopwords)

# transform list to string
df.Text = df.Text.apply(lambda x: ' '.join(x))
df.Text[0]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andre_Rodrigues\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andre_Rodrigues\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better'

##### Normalização das palavras usando a técnica de 'stemming'

In [14]:
def stemming_words(text):
    stemmer = SnowballStemmer('english')
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return stemmed_words

# Normalização das palavras
df.Text = df.Text.apply(stemming_words)

# transform list to string
df.Text = df.Text.apply(lambda x: ' '.join(x))
df.Text[0]

'bought sever vital can dog food product found good qualiti product look like stew process meat smell better labrador finicki appreci product better'

##### Verificar as palavras mais frequentes no dataset


"n" corresponde às top "n" palavras mais frequentes no dataset.

Esta atividade pode nos ajudar a voltar a lista de StopWords e inserir novas palavras para serem excluidas de nossa análise

In [15]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [16]:
# Verificar as 20 palavras mais frequentes no dataset
top_words = get_top_n_words(df.Text, n=20)

In [ ]:
# Printar as 20 palavras mais frequentes
top_df = pd.DataFrame(top_words)
top_df.columns=["Palavra", "Frequência"]
top_df

### Criando "bag of words"

Criaremos agora nossa Bag of Words, que consiste em um conjunto de palavras que existem no nosso Dataset. Cada palavra aparecerá apenas 1x na nossa BOW.

Para isso, iremos realizar a tokenização do nosso dataset. Tokenização consiste em tranformar uma string em uma lista de palavras chamadas tokens. Os tokens podem consistir em palavras, emoticons, hashtags, links ou até mesmo caracteres individuais. Uma maneira básica de dividir as palavras em tokens é dividir o texto com base no espaço em branco e na pontuação.

A biblioteca NLTK fornece um tokenizador padrão com o método word_tokenize(texto)

Com as palavras tokenizadas, iremos colocar uma representação de cada na nossa "bag of words".

In [18]:
# BOW simples com 1 amostra de cada palavra
def generate_bow(sentences):
    # Tokenização/Vetor das palavras
    words = CountVectorizer().fit(sentences)
    # remove duplicates
    words = list(set(words.get_feature_names()))
    return words

# BOW com palavra e frequência
def generate_bow_freq(sentences):
    vec = CountVectorizer().fit(sentences)
    bag_of_words = vec.transform(sentences)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[0:]

# BOW matrix
def generate_bow_matrix(sentences):
    vec = CountVectorizer().fit(sentences)
    bag_of_words = vec.transform(sentences)
    return bag_of_words
    

### Criação do Dataset balanceado

Nosso dataset que será usado para treinar o modelo deverá estar balanceado, ou seja, mesma quantidade de amostras para os 2 grupos de "sentimento"

In [19]:
# verify how many positive and negative reviews we have
df.Sentiment.value_counts()

Positive    3846
Negative     759
Name: Sentiment, dtype: int64

In [20]:
# Temos 759 negativos e 3846 positivos
# Pegamos 759 amostas de cada tipo de sentimento para formar o Dataset de treino (amostras balanceadas)
df_balanced = df.groupby('Sentiment').head(759)
df_balanced.Sentiment.value_counts()

Positive    759
Negative    759
Name: Sentiment, dtype: int64

In [21]:
df_balanced.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,bought sever vital can dog food product found ...,Positive
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arriv label jumbo salt peanutsth peanu...,Negative
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",confect around centuri light pillowi citrus ge...,Positive
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,look secret ingredi robitussin believ found go...,Negative
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,great taffi great price wide assort yummi taff...,Positive


### Criando dataset de treino, validação e teste

In [22]:
# Separando os conjuntos de dados de treino e teste
df_train, df_test, label_column_train, label_column_test = train_test_split(df_balanced, df_balanced['Sentiment'], test_size=0.2, random_state=9)

# print tamanho dos conjuntos de dados
# os conjuntos devem ter o mesmo tamanho
print(f"Train shapes: x = {df_train.shape}, y = {label_column_train.shape}")
print(f"Test shapes: x = {df_test.shape}, y = {label_column_test.shape}")

Train shapes: x = (1214, 11), y = (1214,)
Test shapes: x = (304, 11), y = (304,)


### Constuindo modelos de classificação

Cada modelo receberá o texto como "input" e retornará se é positivo ou negativo

Utilizaremos os modelos para classificação abaixo

#### K-NN model

O KNN (K Nearest Neighbors) é um algoritmo de classificação baseado na ideia de que dados de uma mesma classe estariam próximos entre si no espaço. Para esse cálculo de proximidade, é calculada a distância de um dado não classificado em relação a todos os dados históricos que já possuem classificação. A partir disso, são selecionados os K vizinhos mais próximos do novo dado para determinar a classificação dele via votação. O tipo de classificação que tiver mais vizinhos representantes irá corresponder a classe do novo dado. Esta variável K é a variável mais importante deste algoritmo, e pode ser configurada na variável **K** da função *knn_sentiment_analysis* criada.

Este é um algoritmo não paramétrico que é conhecido por ser lazy, isto é, não necessitar da realização de computação durante o treinamento (apenas exigindo memorização dos dados de treinamento), o que facilita o processo inicial. Por outro lado, isso torna a etapa de predição muito mais custosa e lenta.

In [23]:

# K-nn sentiment analysis model
def generate_knn_model(df, txt_vectorized_arr, k):
    knn = KNeighborsClassifier(n_neighbors=k)

    # Parametros: Textos vetorizados no array 2D, Array de Sentimentos
    knn.fit(txt_vectorized_arr, df.Sentiment.values)

    return knn

# K-nn sentiment analysis of a single dataframe
def knn_sentiment_analysis(df, txt_vectorized_arr, model):
    # cria copia do dataframe para não alterar o original
    df_copy = df.copy()
    # predict
    # Parameters: Textos vetorizados no array 2D
    df_copy['Knn_Predict'] = model.predict(txt_vectorized_arr)

    # check accuracy
    # Parameters: Array de Sentimentos, Array de Sentimentos previstos
    df_copy['Accuracy'] = accuracy_score(df_copy.Sentiment.values, df_copy.Knn_Predict.values)
    return df_copy

In [24]:
cv = CountVectorizer()

# Cria o modelo knn
# Parameters: Dataframe de treino, Array 2D dos textos do DF vetorizados, k-neighbors
# cv.transform(df_train.Text).toarray() gera um Array 2D contendo Vectorized dos textos do nosso DF para que possa ser usado para gerar o modelo
knn_model = generate_knn_model(df_train, cv.fit_transform(df_train.Text).toarray(), 5)


In [25]:
# faz a predição do treino com o modelo criado
df_train_Knn = knn_sentiment_analysis(df_train, cv.transform(df_train.Text).toarray(), knn_model)


In [26]:
df_test_Knn = knn_sentiment_analysis(df_test, cv.transform(df_test.Text).toarray(), knn_model)

##### Resultado Treino

Comparação das quantidades obtidas com o dataset de treino x teste

In [27]:
# Train - Visão geral sobre os sentimentos
print("\nQuantidade de valores positivos e negativos reais no dataset:")
print(df_train_Knn.Sentiment.value_counts())

print("\nQuantidade de valores positivos e negativos encontrados pelo modelo:")
print(df_train_Knn.Knn_Predict.value_counts())


Quantidade de valores positivos e negativos reais no dataset:
Positive    619
Negative    595
Name: Sentiment, dtype: int64

Quantidade de valores positivos e negativos encontrados pelo modelo:
Positive    690
Negative    524
Name: Knn_Predict, dtype: int64


Dataframe de treino

In [28]:
# show dataframe
df_train_Knn.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,Knn_Predict,Accuracy
199,200,B0028C44Z0,A1WTXY4MW3YDF2,Gordon,0,0,5,1344729600,These mints are awesome!,huge suppli im still work plenti spare much ef...,Positive,Negative,0.817957
559,560,B000G6RYNE,A10RJEQN64ATXU,"Paul Rodney Williams ""Higher Lifestyle""",3,3,5,1188777600,delicious,love kettl brand sea salt vinegar chip sinc fi...,Positive,Positive,0.817957
46,47,B001EO5QW8,AQLL2R1PPR46X,grumpyrainbow,0,0,5,1192752000,good,good oatmeal like appl cinnamon best though wo...,Positive,Positive,0.817957
551,552,B000G6RYNE,A2B5OI74EHGVH1,"Jane ""jdeaton2""",3,8,1,1273017600,dripping in oil,purchas low salt inde low salt howev mani mani...,Negative,Negative,0.817957
2466,2467,B002JX7GVM,AN9E7KAWWF95Q,ChristineThomson,0,0,2,1348963200,Terrible flavor,almost gag first tast use coconut base product...,Negative,Negative,0.817957


Dataframe de teste

In [29]:
df_test_Knn.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,Knn_Predict,Accuracy
133,134,B003OB0IB8,AOTEC8KEH8JGN,Seth S Moyers,0,0,5,1334880000,Great value and convenient ramen,got sale rough 25 cent per cup half price loca...,Positive,Negative,0.684211
277,278,B001D07IPG,A3QN14A5DGUA0U,J. Kraayenbrink,0,0,5,1343692800,Excellent for G/F,pleas ignor onestar comment check bag main ing...,Positive,Positive,0.684211
1520,1521,B001LQNX8S,A2QA5W84LGMG8L,Dee,0,0,1,1342742400,These are absolutely revolting,flavor pod horrid order senseo stock long need...,Negative,Positive,0.684211
2951,2952,B000H280KS,AJITVA02GIXPO,Gretchen,0,0,2,1173312000,"Looks better than it tasted, I'm told",sent gift found later mani item stale love tho...,Negative,Positive,0.684211
758,759,B0035YE9CS,AGXFRN1RZKI4J,Earl Hudson,0,1,5,1321228800,Outstanding Product!,saw made interest processdecid give tri realli...,Positive,Negative,0.684211


#### Vader model

Valence Aware Dictionary and sEntiment Reasoner (VADER) é uma biblioteca do Python de código aberto construída para ser usada em tarefas de análise de sentimentos.

A vader funciona de maneira bem simples: ela tem uma coleção de palavras, onde cada palavra já possuí uma nota(ou sentimento) atribuida, e quando passado um documento (frase) retorna os seguintes valores em porcentagem:

        pos: o quão positiva é aquele frase/documento;

        neu: o quão neutra é a frase/documento;

        neg: o quanto é negativa;

compound: é a métrica usada para indicar a conclusão final a respeito da frase como um todo. É calculada somando as pontuações de valência de cada palavra no léxico, o que gera um número entre -1 (muito negativo) e +1 (muito positivo). 

        Se for compound >= 0.05 -> Positivo

        Se for compound <= -0.05 -> Negativo

        Se for compound entre -0.04 e 0.04 -> Neutro

O compound é a métrica mais importante quando você apenas quer saber se aquela frase é positiva ou negativa, porque seu valor pode ser convertido nessas respectivas categorias e é exatamente isso que vamos aprender a fazer aqui!

In [30]:
nltk.download('vader_lexicon')

# Análise completa com valores de sentimento pos, neg, neu e compound
# Retorna um dicionário com os valores de sentimento neg, pos, neu e compound da frase passada
def vader_complete_sentiment_analyse(text):
    sentiment = SentimentIntensityAnalyzer().polarity_scores(text)
    return sentiment

# Análise simples que retorna pos, neg ou neu de acordo com o valor de compound
def vader_get_sentiment(text):
    sentiment = vader_complete_sentiment_analyse(text)
    if sentiment['compound'] >= 0.05:
        return 'Positive'
    elif sentiment['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Análise de sentimento dos DF com o modelo VADER
def vader_df_sentiment_analyse(df):
    # cria copia do dataframe para não alterar o original
    df_copy = df.copy()
    # Sentiment analysis
    df_copy['Vader_Predict'] = df_copy.Text.apply(vader_get_sentiment)
    # check accuracy
    df_copy['Accuracy'] = accuracy_score(df_copy.Sentiment.values, df_copy.Vader_Predict.values)

    return df_copy
    

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Andre_Rodrigues\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [31]:
df_train_vader = vader_df_sentiment_analyse(df_train)
df_test_vader = vader_df_sentiment_analyse(df_test)

##### Resultado Treino

Comparação das quantidades obtidas com o dataset de treino x teste

In [32]:
# Train - Visão geral sobre os sentimentos
print("\nQuantidade de valores positivos e negativos reais no dataset:")
print(df_train_vader.Sentiment.value_counts())

print("\nQuantidade de valores positivos e negativos encontrados pelo modelo:")
print(df_train_vader.Vader_Predict.value_counts())


Quantidade de valores positivos e negativos reais no dataset:
Positive    619
Negative    595
Name: Sentiment, dtype: int64

Quantidade de valores positivos e negativos encontrados pelo modelo:
Positive    946
Negative    190
Neutral      78
Name: Vader_Predict, dtype: int64


In [33]:
df_train_vader.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,Vader_Predict,Accuracy
199,200,B0028C44Z0,A1WTXY4MW3YDF2,Gordon,0,0,5,1344729600,These mints are awesome!,huge suppli im still work plenti spare much ef...,Positive,Positive,0.607908
559,560,B000G6RYNE,A10RJEQN64ATXU,"Paul Rodney Williams ""Higher Lifestyle""",3,3,5,1188777600,delicious,love kettl brand sea salt vinegar chip sinc fi...,Positive,Positive,0.607908
46,47,B001EO5QW8,AQLL2R1PPR46X,grumpyrainbow,0,0,5,1192752000,good,good oatmeal like appl cinnamon best though wo...,Positive,Positive,0.607908
551,552,B000G6RYNE,A2B5OI74EHGVH1,"Jane ""jdeaton2""",3,8,1,1273017600,dripping in oil,purchas low salt inde low salt howev mani mani...,Negative,Negative,0.607908
2466,2467,B002JX7GVM,AN9E7KAWWF95Q,ChristineThomson,0,0,2,1348963200,Terrible flavor,almost gag first tast use coconut base product...,Negative,Negative,0.607908


##### Resultado Teste

In [34]:
# Train - Visão geral sobre os sentimentos
print("\nQuantidade de valores positivos e negativos reais no dataset:")
print(df_train_vader.Sentiment.value_counts())

print("\nQuantidade de valores positivos e negativos encontrados pelo modelo:")
print(df_train_vader.Vader_Predict.value_counts())


Quantidade de valores positivos e negativos reais no dataset:
Positive    619
Negative    595
Name: Sentiment, dtype: int64

Quantidade de valores positivos e negativos encontrados pelo modelo:
Positive    946
Negative    190
Neutral      78
Name: Vader_Predict, dtype: int64


In [35]:
df_test_vader.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,Vader_Predict,Accuracy
133,134,B003OB0IB8,AOTEC8KEH8JGN,Seth S Moyers,0,0,5,1334880000,Great value and convenient ramen,got sale rough 25 cent per cup half price loca...,Positive,Positive,0.546053
277,278,B001D07IPG,A3QN14A5DGUA0U,J. Kraayenbrink,0,0,5,1343692800,Excellent for G/F,pleas ignor onestar comment check bag main ing...,Positive,Positive,0.546053
1520,1521,B001LQNX8S,A2QA5W84LGMG8L,Dee,0,0,1,1342742400,These are absolutely revolting,flavor pod horrid order senseo stock long need...,Negative,Negative,0.546053
2951,2952,B000H280KS,AJITVA02GIXPO,Gretchen,0,0,2,1173312000,"Looks better than it tasted, I'm told",sent gift found later mani item stale love tho...,Negative,Positive,0.546053
758,759,B0035YE9CS,AGXFRN1RZKI4J,Earl Hudson,0,1,5,1321228800,Outstanding Product!,saw made interest processdecid give tri realli...,Positive,Positive,0.546053


#### Regressão Logística model

É um modelo probabilístico de predição que busca prever um valor binário de 0 ou 1, e que é mais maleável e consegue ser utilizado em casos complexos de classificação. Na fórmula deste método, não são considerados outliers, tendo em vista que são apenas considerados dados próximos a uma reta que divide os dados de acordo com os atributos especificados. Assim como o algoritmo Perceptron, é um algoritmo que exige bastante treinamento e que apenas consegue encontrar um separador linear para os dados se estes forem linearmente separáveis.

In [36]:

def logistic_regression_sentiment_analysis(df, iteractions=1000):
    # generate BOW matrix
    bow_matrix = generate_bow_matrix(df.Text)
    # generate BOW frequency
    bow_freq = generate_bow_freq(df.Text)
    # generate BOW frequency dataframe
    bow_freq_df = pd.DataFrame(bow_freq)
    bow_freq_df.columns=["Palavra", "Frequência"]
    
    # Logistic Regression
    lr= LogisticRegression(max_iter=iteractions)
    lr.fit(bow_matrix, df.Sentiment)

    # copy dataframe para não alterar o original
    df_copy = df.copy()
    # predict
    df_copy['Logistic_Predict'] = lr.predict(bow_matrix)
    # check accuracy
    df_copy['Accuracy'] = accuracy_score(df_copy.Sentiment.values, df_copy.Logistic_Predict.values)

    return df_copy

In [37]:
df_train_lr = logistic_regression_sentiment_analysis(df_train)
df_test_lr = logistic_regression_sentiment_analysis(df_test)

##### Resultado Treino

Comparação das quantidades obtidas com o dataset de treino x teste

In [38]:
# Train - Visão geral sobre os sentimentos
print("\nQuantidade de valores positivos e negativos reais no dataset:")
print(df_train_lr.Sentiment.value_counts())

print("\nQuantidade de valores positivos e negativos encontrados pelo modelo:")
print(df_train_lr.Logistic_Predict.value_counts())


Quantidade de valores positivos e negativos reais no dataset:
Positive    619
Negative    595
Name: Sentiment, dtype: int64

Quantidade de valores positivos e negativos encontrados pelo modelo:
Positive    623
Negative    591
Name: Logistic_Predict, dtype: int64


In [39]:
df_train_vader.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,Vader_Predict,Accuracy
199,200,B0028C44Z0,A1WTXY4MW3YDF2,Gordon,0,0,5,1344729600,These mints are awesome!,huge suppli im still work plenti spare much ef...,Positive,Positive,0.607908
559,560,B000G6RYNE,A10RJEQN64ATXU,"Paul Rodney Williams ""Higher Lifestyle""",3,3,5,1188777600,delicious,love kettl brand sea salt vinegar chip sinc fi...,Positive,Positive,0.607908
46,47,B001EO5QW8,AQLL2R1PPR46X,grumpyrainbow,0,0,5,1192752000,good,good oatmeal like appl cinnamon best though wo...,Positive,Positive,0.607908
551,552,B000G6RYNE,A2B5OI74EHGVH1,"Jane ""jdeaton2""",3,8,1,1273017600,dripping in oil,purchas low salt inde low salt howev mani mani...,Negative,Negative,0.607908
2466,2467,B002JX7GVM,AN9E7KAWWF95Q,ChristineThomson,0,0,2,1348963200,Terrible flavor,almost gag first tast use coconut base product...,Negative,Negative,0.607908


##### Resultado Teste

In [40]:
df_test_lr.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,Logistic_Predict,Accuracy
133,134,B003OB0IB8,AOTEC8KEH8JGN,Seth S Moyers,0,0,5,1334880000,Great value and convenient ramen,got sale rough 25 cent per cup half price loca...,Positive,Positive,1.0
277,278,B001D07IPG,A3QN14A5DGUA0U,J. Kraayenbrink,0,0,5,1343692800,Excellent for G/F,pleas ignor onestar comment check bag main ing...,Positive,Positive,1.0
1520,1521,B001LQNX8S,A2QA5W84LGMG8L,Dee,0,0,1,1342742400,These are absolutely revolting,flavor pod horrid order senseo stock long need...,Negative,Negative,1.0
2951,2952,B000H280KS,AJITVA02GIXPO,Gretchen,0,0,2,1173312000,"Looks better than it tasted, I'm told",sent gift found later mani item stale love tho...,Negative,Negative,1.0
758,759,B0035YE9CS,AGXFRN1RZKI4J,Earl Hudson,0,1,5,1321228800,Outstanding Product!,saw made interest processdecid give tri realli...,Positive,Positive,1.0
